In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

plt.style.use("default")
plt.rc("text", usetex=True)
plt.rc("font", family="cm")
plt.rcParams["grid.color"] = (0.5, 0.5, 0.5, 0.2)

In [ ]:
L = 500
x_0 = 200
q = 2
sigma = 20
a = 250
b = 260
V_0 = 1.7
N_x = 2000
dx = L/N_x
N_t = 10000
dt = 0.1
alpha = dt/(2*dx**2)

In [ ]:
# potential barrier
x = np.linspace(0, L, N_x+1)
V_x = np.zeros((N_x+1))
V_x[(x>=a) & (x<=b)] = V_0

In [ ]:
# wave function - initial solution
psi_0 = np.exp(1j*q*x) * np.exp(-(x-x_0)**2/sigma**2)
psi_0 = psi_0 / np.sqrt(np.sum(np.abs(psi_0)**2))

In [ ]:
# crank nicolson matrix (implicit method)
M = np.diag(1 + 2*1j*alpha - 1j*dt*V_x)
M += np.diag(np.full(N_x, 1j*alpha), -1)
M += np.diag(np.full(N_x, 1j*alpha), 1)
M

In [ ]:
# time evolution matrix from t to t+dt
C = 0.5 * np.matmul(sp.linalg.inv(np.eye(N_x+1) - 0.5*M), M)
C

In [ ]:
# wave functions vector
psi = np.zeros((N_x+1, N_t+1), dtype=np.complex128)
psi[:, 0] = psi_0

In [ ]:
for k in range(N_t):
    psi[:, k+1] = np.matmul(C, psi[:,k])
    # normalization
    psi[:, k+1] = psi[:, k+1] / np.sqrt(np.sum(np.abs(psi[:, k+1])**2))

In [ ]:
%%capture
from matplotlib.animation import FuncAnimation, PillowWriter
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,8))

def animate(t): 
    ax.clear()
    ax.plot(x, np.real(psi[:, t*40]), lw=0.75, color='blue', label='Re($\psi$)')
    ax.axvline(a, ls="--", c="red", lw=0.5, ymin=0.5)
    ax.axvline(b, ls="--", c="red", lw=0.5, ymin=0.5)
    ax.set_title('Wave Function Time Evolution')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$\psi \, (x,t)$')
    ax.set_xlim([0, 500])
    ax.set_ylim([-0.3, 0.3])
    ax.grid()
    ax.legend()
       
anim = FuncAnimation(fig, animate, int(N_t/40), blit=False)
anim.save("images/psi_position.gif", writer=PillowWriter(fps=100), dpi=150)

In [ ]:
%%capture
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,8))

def animate(t):
    ax.clear()
    ax.plot(x, np.abs(psi[:, t*40])**2, lw=0.75, color="blue")
    ax.axvline(a, ls="--", c="red", lw=0.5, ymin=0)
    ax.axvline(b, ls="--", c="red", lw=0.5, ymin=0)
    ax.set_title('Probability')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$|\psi \, (x,t)|^2$')
    ax.set_xlim([0, 500])
    ax.set_ylim([0.0, 0.02])
    ax.grid()
       
anim = FuncAnimation(fig, animate, int(N_t/40), blit=False)
anim.save("images/psi_probability.gif", writer=PillowWriter(fps=100), dpi=150)